In [1]:
import threading, time, sys
import rtmidi
import musx
print(musx.version)

2.0.6


In [2]:
midiout = rtmidi.MidiOut()
print(midiout)
outports = midiout.get_ports()
print("available ports:", outports) 

available ports: ['Microsoft GS Wavetable Synth 0', 'charjabug 1', 'mio 2']


In [3]:
loop_midi_outport = 'mio 2'  # find loop midi
midiout.open_port(outports.index(loop_midi_outport))

In [4]:
midiin = rtmidi.MidiIn()
print(midiin)
inports = midiin.get_ports()
print("available input ports:", inports)

available input ports: ['charjabug 0', 'mio 1']


In [5]:
midi_controller_port = 'mio 1'  # find midi controller
midiin.open_port(inports.index(midi_controller_port))

In [6]:
from musx import Pitch, Interval

CUTOFF = Pitch('C4')

def arpeggiator(message, test):
    print(message, test)
    NOTE_INDEX = 1
    VEL_INDEX = 2
    NOTE_ON = 144
    root = message[NOTE_INDEX]
    vel = message[VEL_INDEX]
    is_note_on = message[0] == NOTE_ON

    root_pitch = Pitch.from_keynum(root)

    if root_pitch < CUTOFF:
      return

    chord_intervals = [Interval('P4'), Interval('P5'), Interval('P8')]
    maj_chord = [i.transpose(root_pitch) for i in chord_intervals]

    print(maj_chord)
    
    if is_note_on:
      for pitch in maj_chord:
        midiout.send_message(musx.note_on(0, pitch.keynum(), vel))
        time.sleep(0.14)
        midiout.send_message(musx.note_off(0, pitch.keynum(), vel))
    else:
      for pitch in maj_chord:
        midiout.send_message(musx.note_off(0, pitch.keynum(), 127))

    # time.sleep(.5)

# midiin.set_callback(arpeggiator)

In [8]:
def callback():
  msg = midiin.get_message()
  if msg is not None:
    print(msg)
    t = threading.Thread(target=arpeggiator, args=(msg))
    t.start()

while True:
  callback()

([144, 60, 57], 17.774)
[144, 60, 57] 17.774
[Pitch("F4"), Pitch("G4"), Pitch("C5")]
([176, 64, 61], 0.179)
[176, 64, 61] 0.179
[Pitch("A4"), Pitch("B4"), Pitch("E5")]
([176, 64, 67], 0.008)
[176, 64, 67] 0.008
[Pitch("A4"), Pitch("B4"), Pitch("E5")]
([176, 64, 72], 0.008)
[176, 64, 72] 0.008
[Pitch("A4"), Pitch("B4"), Pitch("E5")]
([176, 64, 76], 0.008)
[176, 64, 76]([176, 64, 80], 0.035)
 0.008
[Pitch("A4"), Pitch("B4"), Pitch("E5")]
[176, 64, 80] 0.035
[Pitch("A4"), Pitch("B4"), Pitch("E5")]
([176, 64, 127], 0.10400000000000001)
[176, 64, 127] 0.10400000000000001
([144, 60, 0], 0.099)[Pitch("A4"), Pitch("B4"), Pitch("E5")]

[144, 60, 0]([176, 64, 81], 0.266) 0.099
[Pitch("F4"), Pitch("G4"), Pitch("C5")]

[176, 64, 81]([176, 64, 75], 0.008) 0.266
[Pitch("A4"), Pitch("B4"), Pitch("E5")]

[176, 64, 75]([176, 64, 71], 0.003)
 0.008[176, 64, 71] 0.003
([176, 64, 65], 0.008)

[Pitch("A4"), Pitch("B4"), Pitch("E5")]
[176, 64, 65] 0.008
[Pitch("A4"), Pitch("B4"), Pitch("E5")]
([176, 64, 60]

KeyboardInterrupt: 